# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.8 MB/s eta 0:00:00


We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU faiss_cpu pymupdf pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 54 (delta 15), reused 20 (delta 7), pack-reused 8
Receiving objects: 100% (54/54), 51.28 MiB | 39.31 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [11]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!


**1. Quantization for Compact Vector Representation
2. Diverse Indexing Structures for Scalability
3. GPU Acceleration for High-Speed Computations
4. Batch Processing for Enhanced Throughput
5. Optimized Distance Computations for Efficiency
6. HNSW Graphs for Fast Nearest Neighbor Search
7. Product Quantization for Balanced Accuracy and Efficiency**

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [12]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [13]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [14]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [15]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [16]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [17]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [18]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Let's test it out!

In [19]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [20]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, and accounting.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [21]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

**To increase diversity and reduce byass**

In [22]:
len(documents)

159

In [23]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

embedding nodes:   0%|          | 0/318 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

**Reasoning: Rewrite the question in a way that enhances the need for reasoning to answer it effectively.**

**Conditioning: Modify the question to introduce a conditional element, which adds complexity to the question.**

**Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.**

Let's look at the output and see what we can learn about it!

In [24]:
testset.test_data[0]

DataRow(question="What did the WIRED investigation from January 2024 find about OpenAI's access to key documents?", contexts=['proceeds going to another charitable enterprise.  \n118. \nA WIRED investigation from January 2024 found that OpenAI has also recently \nshuttered the public’s access to previously available “key documents.” In line with OpenAI’s \noriginal promise of transparency, OpenAI’s IRS documents have indicated since its founding that \nany member of the public may view copies of its governing documents, financial statements, and \nconflict of interest rules. However, when WIRED requested these documents, OpenAI indicated it \nhad changed its policy. Thus, while OpenAI has long touted its commitment to transparency,'], ground_truth="The WIRED investigation from January 2024 found that OpenAI has recently shuttered the public's access to previously available 'key documents'.", evolution_type='simple')

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [25]:
test_df = testset.to_pandas()

In [26]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What did the WIRED investigation from January ...,[proceeds going to another charitable enterpri...,The WIRED investigation from January 2024 foun...,simple,True
1,How did OpenAI's initial work follow in the fo...,[77. \nInitial work at OpenAI followed much in...,OpenAI's initial work followed in the footstep...,simple,True
2,How would IP assets be used to generate profit...,"[business model were valid, it would radically...",The IP assets would be used to enrich the inve...,reasoning,True
3,What strategy video game did OpenAI's team com...,[77. \nInitial work at OpenAI followed much in...,OpenAI's team competed in Dota 2 instead of ch...,reasoning,True
4,What are the benefits of releasing open versio...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,multi_context,True
5,"""What game did OpenAI use reinforcement learni...",[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,multi_context,True
6,"""What was the impact of OpenAI's models on fut...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Their publication did prove to be useful to th...,multi_context,True
7,What concerns do US officials have about OpenA...,[chip fabrication plants. If $10 billion from ...,US officials have concerns about OpenAI's inve...,multi_context,True
8,What were Stephen Hawking's concerns about AGI...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,multi_context,True
9,"""What architecture and pre-training method did...",[those connections to the target language. \n7...,OpenAI used the Generative Pre-Trained Transfo...,reasoning,True


In [27]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [28]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [29]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [30]:
response_dataset[0]

{'question': "What did the WIRED investigation from January 2024 find about OpenAI's access to key documents?",
 'answer': 'The WIRED investigation from January 2024 found that OpenAI had recently shuttered the public\'s access to previously available "key documents."',
 'contexts': ['proceeds going to another charitable enterprise.  \n118. \nA WIRED investigation from January 2024 found that OpenAI has also recently \nshuttered the public’s access to previously available “key documents.” In line with OpenAI’s \noriginal promise of transparency, OpenAI’s IRS documents have indicated since its founding that \nany member of the public may view copies of its governing documents, financial statements, and \nconflict of interest rules. However, when WIRED requested these documents, OpenAI indicated it \nhad changed its policy. Thus, while OpenAI has long touted its commitment to transparency,',
  '1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [31]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [32]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [33]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9392, 'context_recall': 0.9500, 'context_precision': 0.9667, 'answer_correctness': 0.7377}

In [34]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What did the WIRED investigation from January ...,The WIRED investigation from January 2024 foun...,[proceeds going to another charitable enterpri...,The WIRED investigation from January 2024 foun...,1.0,0.984567,1.0,1.000000,0.749485
1,How did OpenAI's initial work follow in the fo...,OpenAI used reinforcement learning to play a g...,[77. \nInitial work at OpenAI followed much in...,OpenAI's initial work followed in the footstep...,1.0,0.861664,1.0,1.000000,0.730519
2,How would IP assets be used to generate profit...,IP assets would be slid into a new for-profit ...,"[business model were valid, it would radically...",The IP assets would be used to enrich the inve...,1.0,0.927805,1.0,0.833333,0.732267
3,What strategy video game did OpenAI's team com...,Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI's team competed in Dota 2 instead of ch...,1.0,0.930192,1.0,1.000000,0.956120
4,What are the benefits of releasing open versio...,The benefits of releasing open versions of pow...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,1.0,0.968113,1.0,1.000000,0.746734
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,1.0,0.927906,0.5,1.000000,0.539319
6,"""What was the impact of OpenAI's models on fut...",The impact of OpenAI's models on future model ...,"[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.0,0.943604,1.0,1.000000,0.839459
7,What concerns do US officials have about OpenA...,US officials are concerned about OpenAI's ties...,[chip fabrication plants. If $10 billion from ...,US officials have concerns about OpenAI's inve...,0.0,0.949040,1.0,1.000000,0.745618
8,What were Stephen Hawking's concerns about AGI...,Stephen Hawking's concerns about AGI in the wr...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.0,0.964592,1.0,0.833333,0.600600
9,"""What architecture and pre-training method did...",OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,OpenAI used the Generative Pre-Trained Transfo...,1.0,0.934339,1.0,1.000000,0.736710


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [35]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [36]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [37]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [38]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [39]:
print(response["answer"])

The plaintiff is Elon Musk.


In [40]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [41]:
print(response["answer"])

The complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices, and seeking accounting, restitution, disgorgement of funds, and an injunction against the Defendants. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [42]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [43]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [44]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [45]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What did the WIRED investigation from January ...,The WIRED investigation from January 2024 foun...,[proceeds going to another charitable enterpri...,The WIRED investigation from January 2024 foun...,1.000000,0.932766,1.0,0.866667,0.615886
1,How did OpenAI's initial work follow in the fo...,OpenAI's initial work followed in the footstep...,[77. \nInitial work at OpenAI followed much in...,OpenAI's initial work followed in the footstep...,0.750000,0.861314,1.0,1.000000,0.610840
2,How would IP assets be used to generate profit...,"If the proposed business model is implemented,...","[business model were valid, it would radically...",The IP assets would be used to enrich the inve...,NaN,0.930065,1.0,1.000000,0.473532
3,What strategy video game did OpenAI's team com...,"OpenAI's team competed in Dota 2, a strategy v...",[77. \nInitial work at OpenAI followed much in...,OpenAI's team competed in Dota 2 instead of ch...,1.000000,0.965095,1.0,1.000000,0.996399
4,What are the benefits of releasing open versio...,The benefits of releasing open versions of pow...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,1.000000,0.968113,1.0,1.000000,0.808760
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning in the stra...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,0.500000,0.937840,0.5,1.000000,0.615114
6,"""What was the impact of OpenAI's models on fut...",The impact of OpenAI's models on future model ...,"[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.000000,0.939927,1.0,1.000000,0.778422
7,What concerns do US officials have about OpenA...,US officials have concerns about OpenAI's ties...,[chip fabrication plants. If $10 billion from ...,US officials have concerns about OpenAI's inve...,0.000000,0.966597,1.0,1.000000,0.490880
8,What were Stephen Hawking's concerns about AGI...,"Stephen Hawking, along with other luminaries l...",[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.000000,0.924122,1.0,1.000000,0.528843
9,"""What architecture and pre-training method did...",OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,OpenAI used the Generative Pre-Trained Transfo...,0.333333,0.916040,1.0,0.950000,0.668956


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [46]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9392, 'context_recall': 0.9500, 'context_precision': 0.9667, 'answer_correctness': 0.7377}

And see how our advanced retrieval modified our chain!

In [47]:
advanced_retrieval_results

{'faithfulness': 0.7315, 'answer_relevancy': 0.9342, 'context_recall': 0.9500, 'context_precision': 0.9817, 'answer_correctness': 0.6588}

In [48]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.900000,0.731481,-0.168519
1,answer_relevancy,0.939182,0.934188,-0.004994
2,context_recall,0.950000,0.950000,0.000000
3,context_precision,0.966667,0.981667,0.015000
4,answer_correctness,0.737683,0.658763,-0.078920


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

**Create embedding from OpenAI´s text-embedding-3-small**

In [49]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

**Create FAISS index**

In [50]:
vector_store = FAISS.from_documents(documents, new_embeddings)

**Create retriever from vector store**

In [51]:
new_retriever = vector_store.as_retriever()

**Create a different (more advanced) retriever**

In [52]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

**Create retrieval chain for the advanced retriever**

In [53]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

**Capture answers and references used by the model when infering user´s questions**

In [54]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

**Output fomatting**

In [55]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

**Use RAGAS for evaluation**

In [56]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

**Print evaluation results**

In [57]:
new_advanced_retrieval_results

{'faithfulness': 0.9537, 'answer_relevancy': 0.9322, 'context_recall': 0.9250, 'context_precision': 0.9317, 'answer_correctness': 0.7102}

**Build dataframe to visualize results from different embeddings**

In [58]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.900000,0.731481,0.953704,0.222222,0.053704
1,answer_relevancy,0.939182,0.934188,0.932174,-0.002014,-0.007008
2,context_recall,0.950000,0.950000,0.925000,-0.025000,-0.025000
3,context_precision,0.966667,0.981667,0.931667,-0.050000,-0.035000
4,answer_correctness,0.737683,0.658763,0.710160,0.051396,-0.027524


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?
**It depends on the application. TE3 performs better in two of the metrics (faithfulness and answer correctness) but works slightly worse in the others (specially context precision). There is a tradeoff is between faithfulness and context precision)**

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [59]:
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What did the WIRED investigation from January ...,The WIRED investigation from January 2024 foun...,[proceeds going to another charitable enterpri...,The WIRED investigation from January 2024 foun...,1.000000,0.932766,1.0,0.866667,0.615886
1,How did OpenAI's initial work follow in the fo...,OpenAI's initial work followed in the footstep...,[77. \nInitial work at OpenAI followed much in...,OpenAI's initial work followed in the footstep...,0.750000,0.861314,1.0,1.000000,0.610840
2,How would IP assets be used to generate profit...,"If the proposed business model is implemented,...","[business model were valid, it would radically...",The IP assets would be used to enrich the inve...,NaN,0.930065,1.0,1.000000,0.473532
3,What strategy video game did OpenAI's team com...,"OpenAI's team competed in Dota 2, a strategy v...",[77. \nInitial work at OpenAI followed much in...,OpenAI's team competed in Dota 2 instead of ch...,1.000000,0.965095,1.0,1.000000,0.996399
4,What are the benefits of releasing open versio...,The benefits of releasing open versions of pow...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,1.000000,0.968113,1.0,1.000000,0.808760
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning in the stra...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,0.500000,0.937840,0.5,1.000000,0.615114
6,"""What was the impact of OpenAI's models on fut...",The impact of OpenAI's models on future model ...,"[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.000000,0.939927,1.0,1.000000,0.778422
7,What concerns do US officials have about OpenA...,US officials have concerns about OpenAI's ties...,[chip fabrication plants. If $10 billion from ...,US officials have concerns about OpenAI's inve...,0.000000,0.966597,1.0,1.000000,0.490880
8,What were Stephen Hawking's concerns about AGI...,"Stephen Hawking, along with other luminaries l...",[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.000000,0.924122,1.0,1.000000,0.528843
9,"""What architecture and pre-training method did...",OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,OpenAI used the Generative Pre-Trained Transfo...,0.333333,0.916040,1.0,0.950000,0.668956


In [62]:
new_advanced_retrieval_results_df = new_advanced_retrieval_results.to_pandas()
new_advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What did the WIRED investigation from January ...,The WIRED investigation from January 2024 foun...,[proceeds going to another charitable enterpri...,The WIRED investigation from January 2024 foun...,1.000000,0.984576,1.00,0.833333,0.610023
1,How did OpenAI's initial work follow in the fo...,OpenAI's initial work followed in the footstep...,[77. \nInitial work at OpenAI followed much in...,OpenAI's initial work followed in the footstep...,NaN,0.868026,1.00,1.000000,0.735684
2,How would IP assets be used to generate profit...,"If the proposed business model is implemented,...","[business model were valid, it would radically...",The IP assets would be used to enrich the inve...,0.750000,0.888305,1.00,0.700000,0.890990
3,What strategy video game did OpenAI's team com...,"OpenAI's team competed in Dota 2, a strategy v...",[77. \nInitial work at OpenAI followed much in...,OpenAI's team competed in Dota 2 instead of ch...,1.000000,0.943860,1.00,1.000000,0.996415
4,What are the benefits of releasing open versio...,The benefits of releasing open versions of pow...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,1.000000,0.970443,1.00,1.000000,0.746710
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning in the stra...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,1.000000,0.929121,0.75,1.000000,0.535378
6,"""What was the impact of OpenAI's models on fut...","OpenAI's models, particularly the ""Transformer...","[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.000000,0.939927,1.00,0.950000,0.775970
7,What concerns do US officials have about OpenA...,US officials are concerned about OpenAI's ties...,[chip fabrication plants. If $10 billion from ...,US officials have concerns about OpenAI's inve...,0.833333,0.959734,1.00,0.966667,0.540733
8,What were Stephen Hawking's concerns about AGI...,"Stephen Hawking, along with luminaries like Bi...",[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.000000,0.891366,1.00,1.000000,0.602994
9,"""What architecture and pre-training method did...",OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,OpenAI used the Generative Pre-Trained Transfo...,1.000000,0.946380,0.50,0.866667,0.666698
